In [28]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/image-people/954471/Train/F/121282643_1739796282838922_3924762867765759060_n.jpg
/kaggle/input/image-people/954471/Train/F/121597221_972711616544097_204099069773999800_o.jpg
/kaggle/input/image-people/954471/Train/F/121469650_1012059645926928_9115015357761950295_o.jpg
/kaggle/input/image-people/954471/Train/F/121453866_1739796116172272_6350458709778971249_n.jpg
/kaggle/input/image-people/954471/Train/F/121256709_972711336544125_7929623606048122247_o.jpg
/kaggle/input/image-people/954471/Train/F/121526847_972713809877211_481376914472549411_o.jpg
/kaggle/input/image-people/954471/Train/F/121460887_1012057685927124_1603928742736483276_o.jpg
/kaggle/input/image-people/954471/Train/F/121465524_1012059792593580_5614978863786312552_o.jpg
/kaggle/input/image-people/954471/Train/M/121222913_2888173951406016_2943121663743613328_n.jpg
/kaggle/input/image-people/954471/Train/M/121335583_2114014788729050_910466790392877863_n.jpg
/kaggle/input/image-people/954471/Train/M/121368482_3314

In [29]:
#การตั้งชื้อโฟเดอร์ให้ตรงกับdataของเรา
test_dir="/kaggle/input/image-people/954471/Test"
train_dir="/kaggle/input/image-people/954471/Train"

train_dir_F = train_dir + '/F'
train_dir_M = train_dir + '/M'
test_dir_F = test_dir + '/F'
test_dir_M = test_dir + '/M'

In [30]:
print('number of F training images - ',len(os.listdir(train_dir_F)))
print('number of M training images - ',len(os.listdir(train_dir_M)))
print('number of F testing images - ',len(os.listdir(test_dir_F)))
print('number of M testing images - ',len(os.listdir(test_dir_M)))

number of F training images -  8
number of M training images -  8
number of F testing images -  4
number of M testing images -  4


In [31]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
data_generator = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        rescale = 1.0/255.0,   # Intensity Normalized
        zca_whitening=False,  # apply ZCA whitening ช่วยเรื่องแสง
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180) บิดภาพตั้งแต่0-10(บิดภาพซ้ายขวา)
        zoom_range = 0.2, # Randomly zoom image 
        #width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width) #การฉีกภาพ
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False,   # randomly flip images
        validation_split=0.2) #แบ่งข้อมูลtrain test

In [123]:
batch_size = 1 #เอารูปให้โมเดลเรียนครั้งละเท่าไหร่ ต้องมีจำนวนน้อยหรือเท่ากับจำนวนรูป
img_height = 96 
img_width  = 96
train_generator = data_generator.flow_from_directory(
    train_dir, #ตัวที่setไว้ด้านบนว่ามันอยู่ตรงไหน
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary', #ปัญหามีแค่2classเลยเป็นbinary
    subset='training') # set as training data บอกว่า dataset ชุดนี้เป็นข้อมูลแบบไหน

validation_generator = data_generator.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation') # set as validation data

Found 14 images belonging to 2 classes.
Found 2 images belonging to 2 classes.


In [124]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
# preparing the layers in the Convolutional Deep Neural Network
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = train_generator.image_shape)) #ใช้32ฟิวเตอร์
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.3)) #rateในการดรอป
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.2))
model.add(Conv2D(filters = 126, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.15))
model.add(Flatten()) #ทำให้กลายเป็นเฟคเตอร์1มิติ และโค้ดข้างล่างคือโค้ดการทำนาย
model.add(Dense(units = 32, activation = 'relu')) 
model.add(Dropout(rate = 0.15))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(rate = 0.1))
model.add(Dense(units = 1, activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [125]:
train_generator.class_indices #เช็คว่าclass0และ1คืออะไร

{'F': 0, 'M': 1}

In [126]:
fitted_model = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = 20)  #ทำ20รอบ dataถูกaugamentation 20 รอบ

Epoch 1/20
14/14 [==============================] - 0s 30ms/step - loss: 0.9021 - accuracy: 0.5000 - val_loss: 0.6946 - val_accuracy: 0.5000
Epoch 2/20
14/14 [==============================] - 0s 20ms/step - loss: 0.6923 - accuracy: 0.5000 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 3/20
14/14 [==============================] - 0s 20ms/step - loss: 0.6981 - accuracy: 0.2143 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 4/20
14/14 [==============================] - 0s 19ms/step - loss: 0.6935 - accuracy: 0.4286 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 5/20
14/14 [==============================] - 0s 20ms/step - loss: 0.6880 - accuracy: 0.7143 - val_loss: 0.6912 - val_accuracy: 1.0000
Epoch 6/20
14/14 [==============================] - 0s 20ms/step - loss: 0.6845 - accuracy: 0.6429 - val_loss: 0.6824 - val_accuracy: 0.5000
Epoch 7/20
14/14 [==============================] - 0s 20ms/step - loss: 0.6703 - accuracy: 0.5000 - val_loss: 0.6583 - val_accuracy: 1.0000
Epoch 8/20
14

In [143]:
from tensorflow.keras.preprocessing import image
# testing the model

test_image = image.load_img('../input/image-people/954471/Test/F/121297902_1739796202838930_2202079621057532442_n.jpg', target_size = (img_height, img_width)) 
test_image = image.img_to_array(test_image) #แปลงค่าเป็นnumpy array
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(np.round(result[0]))

[0.]


In [136]:
from tensorflow.keras.preprocessing import image
# testing the model

test_image = image.load_img('../input/image-people/954471/Test/F/121558240_972714016543857_6033716185027836421_o.jpg', target_size = (img_height, img_width)) 
test_image = image.img_to_array(test_image) #แปลงค่าเป็นnumpy array
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(np.round(result[0]))

[0.]


In [137]:
from tensorflow.keras.preprocessing import image
# testing the model

test_image = image.load_img('../input/image-people/954471/Test/F/121638990_1012059519260274_2330212418373086362_o.jpg', target_size = (img_height, img_width)) 
test_image = image.img_to_array(test_image) #แปลงค่าเป็นnumpy array
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(np.round(result[0]))

[0.]


In [138]:
from tensorflow.keras.preprocessing import image
# testing the model

test_image = image.load_img('../input/image-people/954471/Test/F/121711378_1739798592838691_2434137178508018006_n.jpg', target_size = (img_height, img_width)) 
test_image = image.img_to_array(test_image) #แปลงค่าเป็นnumpy array
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(np.round(result[0]))

[0.]


In [139]:
from tensorflow.keras.preprocessing import image
# testing the model

test_image = image.load_img('../input/image-people/954471/Test/M/121160480_3678313258869851_5818106878740531607_o.jpg', target_size = (img_height, img_width)) 
test_image = image.img_to_array(test_image) #แปลงค่าเป็นnumpy array
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(np.round(result[0]))

[1.]


In [140]:
from tensorflow.keras.preprocessing import image
# testing the model

test_image = image.load_img('../input/image-people/954471/Test/M/121218785_2888174644739280_5191976153731649311_n.jpg', target_size = (img_height, img_width)) 
test_image = image.img_to_array(test_image) #แปลงค่าเป็นnumpy array
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(np.round(result[0]))

[1.]


In [141]:
from tensorflow.keras.preprocessing import image
# testing the model

test_image = image.load_img('../input/image-people/954471/Test/M/121558243_3459617724076248_4826110803681428223_n.jpg', target_size = (img_height, img_width)) 
test_image = image.img_to_array(test_image) #แปลงค่าเป็นnumpy array
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(np.round(result[0]))

[1.]


In [150]:
from tensorflow.keras.preprocessing import image
# testing the model

test_image = image.load_img('../input/image-people/954471/Test/M/121623008_2688765031372476_1265210388791973144_n.jpg', target_size = (img_height, img_width)) 
test_image = image.img_to_array(test_image) #แปลงค่าเป็นnumpy array
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(np.round(result[0]))

[1.]
